<a href="https://colab.research.google.com/github/pietrodileo/Python_for_MD_thesis/blob/main/GeneticAlgorithmWrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
from scipy.stats import pointbiserialr
from math import sqrt
from sklearn.feature_selection import SelectKBest
import random
import time
import random
import pandas as pd
import numpy as np
import copy
import math
import tensorflow
import sklearn
import keras

from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

from sklearn.metrics import accuracy_score
%matplotlib inline

In [ ]:
#Here's a function for DataFrames (again from SO):

from collections import defaultdict

def reverse_dummy(df_dummies):
    pos = defaultdict(list)
    vals = defaultdict(list)

    for i, c in enumerate(df_dummies.columns):
        if "_" in c:
            k, v = c.split("_", 1)
            pos[k].append(i)
            vals[k].append(v)
        else:
            pos["_"].append(i)

    df = pd.DataFrame({k: pd.Categorical.from_codes(
                              np.argmax(df_dummies.iloc[:, pos[k]].values, axis=1),
                              vals[k])
                      for k in vals})

    df[df_dummies.columns[pos["_"]]] = df_dummies.iloc[:, pos["_"]]
    return df

In [ ]:
from keras import backend as K

def recall_fun(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_fun(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_fun(y_true, y_pred):
    precision = precision_fun(y_true, y_pred)
    recall = recall_fun(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def generate_dataframes_for_training(dataframe, split_frac=0.8):
    """
    Generates training and testing dataframes from a complete dataframe, according to the split_frac parameter
    """
    train_data = dataframe.sample(frac=split_frac, random_state=314)
    test_data = dataframe.drop(train_data.index)
    
    X_train = train_data.loc[:, train_data.columns != 'Class']
    X_test = test_data.loc[:, test_data.columns != 'Class']
    y_train = train_data.loc[:, train_data.columns == 'Class']
    y_test = test_data.loc[:, test_data.columns == 'Class']

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    return X_train, X_test, y_train, y_test

In [ ]:
# GETTER FOR UNIQUE VALUES
def get_uniques(dataframe):
    for feature in dataframe.columns:

        print(f"Feature {feature}: \n{list(pd.Series(dataframe[feature]).unique())}")

In [ ]:
# GETTER FOR UNIQUE VALUES
def get_uniques(dataframe):
    for feature in dataframe.columns:

        print(f"Feature {feature}: \n{list(pd.Series(dataframe[feature]).unique())}")

## Functions for Genetic Algorithms

In [ ]:
def create_model(X, verbose=False):
    """
    X: training dataset to be used. Its shape is used to set the input shape for the model.
    """
    
    model = Sequential()
    
    model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    if verbose: print('MODEL SUMMARY: \n')
    if verbose: print(model.summary())
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [ ]:
def get_n_individual(counter, population):
    """
    If counter is 0, return the individual with the highest prob
    If counter is 1, return the second individual with the highest prob
    If counter is 2, return the third individual withthe highest prob
    """
    index = counter + 1
    probabilities = [ind[1] for ind in population]
    sorted_probs = sorted(probabilities, key=float)
    max_prob = probabilities[-index]
    max_individual = [ind[0] for ind in population if ind[1] == max_prob][0]
    
    return max_individual



First, randomly define N possible solutions to the problem

In [ ]:
def generate_random_individuals(num_individuals, num_features, max_features=None, verbose=False):
    """
    Randomly generates individuals

    The number of individuals to generate is given by the num_individuals parameter
    The length of each individual is equal to the num_features parameter
    The maximum number of active features for every individual is given by the max_features parameter
    """
    if verbose: print('GENERATING RANDOM INDIVIDUALS.... ')
        
    individuals = list()
    
    for _ in range(num_individuals):
        individual = ''
        for col in range(num_features):
            # For each char in the individual, a 1 or a 0 is randomly generated
            if individual.count('1') == max_features:
                individual += '0'
                continue
                
            individual += str(random.randint(0, 1))
            
        if verbose: print(f'Generated a new individual: {individual}')
        individuals.append(individual)
        
    if verbose: print(f'Generated list of {num_individuals} individuals: {individuals}')
        
    return individuals

Fitness Evaluation of the current individual (= set of features)

Fitness is based on a neural network

In [ ]:
def get_fitness_func(individual, dataframe, verbose=False):
    """
    Calculate accuracy for the individual passed as parameter.
    Both the dataframe and the y_data parameters are used for training and evaluating the model.
    """
    if verbose: print('Calculating accuracy for individual ', individual)
    
    # generate_dataframes_for_training() function splits a given dataset into training and test data, 
    # and separates labels and rest of features
    X_train, X_test, y_train, y_test = generate_dataframes_for_training(dataframe)
    
    X_train = X_train.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
    X_test = X_test.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]    
    
    model = create_model(X_train, individual)

    X_train = np.asarray(X_train).astype(np.float64)
    X_test = np.asarray(X_test).astype(np.float64)
    y_train = np.asarray(y_train).astype(np.float64)
    y_test = np.asarray(y_test).astype(np.float64)

    model.fit(X_train, y_train, epochs=100, verbose=0)
    
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred.round())
    if verbose: print(f"Accuracy for the classifier trained for individual {individual}: \n", accuracy,'\n')
    
    return accuracy


Get Weights of the current population (needed in reproduction step)

In [ ]:
# This function perform Roulette Wheel Selection
def get_weights_wheel(population):
    """
    Calculate weights from the population filled with the accuracies
    """
    total_accuracies = 0
    new_population = []
    
    # Get the sum of all accuracies of the population
    for individual in population:
        total_accuracies += individual[1]
        
    # For each individual, calculate its weight by dividing its accuracy by the overall sum calculated above
    for individual in population:
        weight = individual[1]/total_accuracies
        # Store the individual and its weight in the final population list
        new_population.append((individual[0], float(weight*100)))
        
    return new_population

# This function perform Rank Selection
def get_weights_rank(population):
    """
    Calculate weights from the population filled with the accuracies
    """
    new_population = []
    N = len(population[0])
    # Use the gauss formula to get the sum of all ranks (sum of integers 1 to N).
    TotalFitness = (N + 1) * N / 2;

    # For each individual, calculate its weight by dividing its accuracy by the overall sum calculated above
    for individual in population:
        weight = individual/TotalFitness
        # Store the individual and its weight in the final population list
        new_population.append((individual[0], float(weight*100)))
        
    return new_population

Evaluate all the individuals one by one

In [ ]:
def fill_population(individuals, dataframe, verbose=False):
    """
    Fills the population list with individuals and their weights
    """
    population = list()
    
    for individual in individuals:
        
        # Get the value of the fitness function (accuracy of the model)
        if verbose: 
          print(f'\n Calculating fitness function value for individual {individual} \n')
        
        # Evaluate current individual
        accuracy = get_fitness_func(individual, dataframe, verbose)
        
        # Check that the value is not the goal state (in this case, an accuracy of 80% is a terminal state)
        if float(accuracy) > 0.8:
            if verbose: print(f'Goal state found for individual {individual}')
            return individual
            
        individual_complete = (individual, accuracy)
        population.append(individual_complete)
        
    # The final population list is created, which contains each individual together with its weight
    # (weights will be used in the reproduction step)
    new_population = get_weights_rank(population)
    # Alternative: roulette wheel selection
    #new_population = get_weights_wheel(population)
    if verbose: print(f'Generated population list (with weights): {new_population}')
    
    return new_population

Choose parents and reproduce them

In [ ]:
def choose_parents(population, counter):
    """
    From the population, weighting the probabilities of an individual being chosen via the fitness
    function, takes randomly two individual to reproduce
    Population is a list of tuples, where the first element is the individual and the second
    one is the probability associated to it.
    To avoid generating repeated individuals, 'counter' parameter is used to pick parents in different ways, thus
    generating different individuals
    """
    # Pick random parent Number 1 and Number 2
    # (get_n_individual() function randomly picks an individual following the distribution of the weights)
    if counter == 0:        
        parent_1 = get_n_individual(0, population)        
        parent_2 = get_n_individual(1, population)
    elif counter == 1:
        parent_1 = get_n_individual(0, population)        
        parent_2 = get_n_individual(2, population)
        
    else:
        probabilities = (individual[1] for individual in population)
        individuals = [individual[0] for individual in population]
        parent_1, parent_2 = random.choices(individuals, weights=probabilities, k=2)
    
    return [parent_1, parent_2]

def mutate(child, prob=0.1):
    """
    Randomly mutates an individual according to the probability given by prob parameter
    """
    new_child = copy.deepcopy(child)
    for i, char in enumerate(new_child):
        if random.random() < prob:
            new_value = '1' if char == '0' else '0'
            new_child = new_child[:i] + new_value + new_child[i+1:]
    
    return new_child

def reproduce(individual_1, individual_2):
    """
    Takes 2 individuals, and combines their information based on a
    randomly chosen crosspoint.
    Each reproduction returns 2 new individuals
    """ 
    # Randomly generate a integer between 1 and the length of the individuals minus one, which will be the crosspoint
    crosspoint = random.randint(1, len(individual_1)-1)
    child_1 = individual_1[:crosspoint] + individual_2[crosspoint:]
    child_2 = individual_2[:crosspoint] + individual_1[crosspoint:]
    child_1, child_2 = mutate(child_1), mutate(child_2)
 
    return [child_1, child_2]

def generation_ahead(population, verbose=False):
    """
    Reproduces all the steps for choosing parents and making 
    childs, which means creating a new generation to iterate with
    """
    new_population = list()
    
    for _ in range(int(len(population)//2)):      
        # According to the weights calculated before, choose a set of parents to reproduce
        parents = choose_parents(population, counter=_)
        if verbose: print(f'Parents chosen: {parents}')
          
        # Reproduce the pair of individuals chose above to generate two new individuals
        childs = reproduce(parents[0], parents[1])
        if verbose: print(f'Generated children: {childs}\n')
        new_population += childs
        
    return new_population

Main Loop

In [ ]:
def main_loop(ind_num, dataframe, max_iter=5, verbose=False):
    """
    Performs all the steps of the Genetic Algorithm
    1. Generate random population
    2. Fill population with the weights of each individual
    3. Check if the goal state is reached
    4. Reproduce the population, and create a new generation
    5. Repeat process until termination condition is met
    """
    num_features = len(dataframe.columns)-1
    # Generate individuals (returns a list of strings, where each str represents an individual)
    individuals = generate_random_individuals(ind_num, num_features, random.randint(0, num_features), verbose)
    
    # Returns a list of tuples, where each tuple represents an individual and its weight
    population = fill_population(individuals, dataframe, verbose)
    
    # Check if a goal state is reached
    # When goal state is reached, fill_population() function returns a str, otherwise continue
    if isinstance(population, str):
        # Evaluate current population
        accuracy = get_fitness_func(population, dataframe, verbose)
        return population, accuracy
        
    # Reproduce current generation to generate a better new one
    new_generation = generation_ahead(population, verbose)
    
    # After the new generation is generated, the loop goes on until a solution is found or until the maximum number of
    # iterations are reached
    iteration_count = 0
    while iteration_count < max_iter:
        if verbose: print(f'\n\n\nITERATION NUMBER {iteration_count+1} (Iteration max = {max_iter+1})\n\n\n')
        population = fill_population(new_generation, dataframe, verbose)
        
        # Check if a goal state is reached
        if isinstance(population, str):
            break
        
        new_generation = generation_ahead(population, verbose)   
        iteration_count += 1
        
    return population


Feature Selection

In [ ]:
# load the dataset
task = 'C14'
excelName = '/content/drive/MyDrive/TesiMagistrale/ExcelForTraining'+'/'+task+'.xlsx'
df = pd.read_excel(excelName)
outpath = '/content/drive/MyDrive/TesiMagistrale/ExcelForTraining/RisultatiFeatureSelection'
# Replace NaN with zero on all columns 
df = df.fillna(0)

# name of the label (can be seen in the dataframe)
label = 'Class'
istance = 'Istance'

# list with feature names (V1, V2, V3, ...)
features = df.columns.tolist()
# keep only numerical values
features.remove(label)
features.remove(istance)

# make a label dataframe
labelVec = df.Class
# create dummy variables
labelDummy = pd.get_dummies(labelVec)
labelDummy.columns = list(range(0, len(labelDummy.columns)))

# reverseDummy: change class labeling to 0 and 1
ClassNumerical = pd.DataFrame(labelDummy.idxmax(axis=1))
ClassNumerical.columns = ['Class']
# convert dataframe to series
ClassNumerical = ClassNumerical.squeeze()

In [ ]:
import scipy.stats as stats

# Define Dataframe
dfGA = df.iloc[: , 1:]
# Calculate the z-score from with scipy
dfGA = dfGA.select_dtypes(include='number').apply(stats.zscore)
dataframe = pd.concat([dfGA, pd.DataFrame(ClassNumerical)], axis=1)

In [ ]:
# # Create the model
# model = Sequential()
# model.add(Dense(64, input_dim=dfGA.shape[1], activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
  
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# # Generate dataframes for both training and testing
# X_train, X_test, y_train, y_test = generate_dataframes_for_training(dataframe)
# X_train = np.asarray(X_train).astype('float32')
# X_test = np.asarray(X_test).astype('float32')
# y_train = np.asarray(y_train).astype('float32')
# y_test = np.asarray(y_test).astype('float32')

In [ ]:
# start = time.time()
# model.fit(X_train, y_train, epochs=128, verbose=0)
# print('Time elapsed for training the model with the full dataset: ', time.time() - start, ' seconds')

In [ ]:
# pred = model.predict(X_test)
# print('Recall: ', recall_fun(y_test, pred.round()))
# print('Precision: ', precision_fun(y_test, pred.round()))
# print('F1 Score: ', f1_fun(y_test, pred.round()))

In [ ]:
start = time.time()
final_population, fitness_accuracy = main_loop(200, dataframe, max_iter = 10, verbose=True)
print('Time elapsed for executing the recursive GA: ', time.time() - start, ' seconds')

In [ ]:
featuresGAall = pd.DataFrame(features).transpose()
selected_featuresGA = featuresGAall.loc[:, [True if char == '1' else False for char in final_population]]
selected_featuresGA

In [ ]:
optimized_dataframe = dataframe.loc[:, [True if char == '1' else False for char in final_population+'0']]
optimized_dataframe['Class'] = dataframe['Class']

# Drop last column
optimized_dataframe.drop(columns=df.columns[-1], 
        axis=1, 
        inplace=True)

importances = mutual_info_classif(optimized_dataframe, ClassNumerical)

mutual_info = pd.Series(importances)
mutual_info.index = optimized_dataframe.columns
mutual_info.sort_values(ascending=False, inplace = True)
k = 100
bestkfeatures = mutual_info[0:k]

feat_importances = pd.Series(bestkfeatures, optimized_dataframe.columns[0:len(optimized_dataframe.columns)])
feat_importances.plot(kind='barh',color = 'teal')
plt.show()

In [ ]:
infogainGA = pd.DataFrame(feat_importances)
infogainGA.columns = ['value']
infogainGA.sort_values(by=['value'],ascending=False,inplace=True)
infogainGA = infogainGA.transpose()
infogainGA.transpose()
infogainGA

In [ ]:
optimized_dataframe['Class'] = dataframe['Class']
outname = task+'_GA_IG.xlsx'
outname2 = task+'_GA_IG_Rank.xlsx'
outGA = os.path.join(outpath,outname)
outGA2 = os.path.join(outpath,outname2)
optimized_dataframe.to_excel(outGA)
infogainGA.to_excel(outGA2)

In [ ]:
# optimized_dataframe = dataframe.loc[:, [True if char == '1' else False for char in final_population+'0']]
# optimized_dataframe['Class'] = dataframe['Class']
# # Create the model
# model = Sequential()
# model.add(Dense(64, input_dim=optimized_dataframe.shape[1], activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
  
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# # Generate dataframes for both training and testing
# X_train, X_test, y_train, y_test = generate_dataframes_for_training(optimized_dataframe)
# X_train = np.asarray(X_train).astype('float32')
# X_test = np.asarray(X_test).astype('float32')
# y_train = np.asarray(y_train).astype('float32')
# y_test = np.asarray(y_test).astype('float32')

In [ ]:
# start = time.time()
# model.fit(X_train, y_train, epochs=128, verbose=0)
# print('Time elapsed for training the model with the full dataset: ', time.time() - start, ' seconds')

In [ ]:
# print('Recall: ', recall_fun(y_test, pred.round()))
# print('Precision: ', precision_fun(y_test, pred.round()))
# print('F1 Score: ', f1_fun(y_test, pred.round()))